### Langchain
[LangChain](https://python.langchain.com/en/latest/index.html) is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model, but will also be:
- Data-aware: connect a language model to other sources of data
- Agentic: allow a language model to interact with its environment

The LangChain framework is designed around these principles.

We will use Langchain framework for rest of the workshop.

#### Question Answering over the docs/index
Question answering in this context refers to question answering over your document data.  For question answering over many documents, you almost always want to create an index over the data. This can be used to smartly access the most relevant documents for a given question, allowing you to avoid having to pass all the documents to the LLM (saving you time and money).

#### Set Environment Variables

In [1]:
import json  
from openai import OpenAI, AzureOpenAI, AsyncAzureOpenAI
from Utilities.envVars import *
import os

os.environ["AZURESEARCH_FIELDS_ID"] = "id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "content"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "contentVector"
os.environ["AZURESEARCH_FIELDS_TAG"] = "{}"

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

azure_endpoint  = f"{OpenAiEndPoint}"
api_key = OpenAiKey
api_version = OpenAiVersion

client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    #base_url=f"{os.getenv('OpenAiWestUsEp')}openai/deployments/{os.getenv('OpenAiGpt4v')}/extensions",
    azure_endpoint=azure_endpoint,
)

#### Generate answer for a question from the document we already indexed in Vector Store

In [2]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from Utilities.cogSearch import performCogSearch
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
        llm = AzureChatOpenAI(
                azure_endpoint=OpenAiEndPoint,
                api_version=OpenAiVersion,
                azure_deployment=OpenAiChat,
                temperature=temperature,
                api_key=OpenAiKey,
                openai_api_type="azure",
                max_tokens=tokenLength)

        embeddings = AzureOpenAIEmbeddings(azure_endpoint=OpenAiEndPoint, azure_deployment=OpenAiEmbedding, api_key=OpenAiKey, openai_api_type="azure")
        logging.info("LLM Setup done")
elif embeddingModelType == "openai":
        llm = ChatOpenAI(temperature=temperature,
        openai_api_key=OpenAiApiKey,
        model_name="gpt-3.5-turbo",
        max_tokens=tokenLength)
        embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)

c:\Users\astalati\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [3]:
# We already created our index and loaded the data, so we can skip that part. Let's try to ask a question:
# Question answering involves fetching multiple documents, and then asking a question of them. 
# The LLM response will contain the answer to your question, based on the content of the documents.
# The simplest way of using Langchain and LLM is to use load_qa_chain and run it with a query and a list of documents.

chainType = "stuff"
topK = 3
query = "What is Microsoft Fabric"

# Since we already index our document, we can perform the search on the query to retrieve "TopK" documents
r = performCogSearch(OpenAiEndPoint, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey, embeddingModelType, OpenAiEmbedding, query, indexName, topK)

if r == None:
    docs = [Document(page_content="No results found")]
else :
    docs = [
        Document(page_content=doc['content'], metadata={"id": doc['id'], "source": doc['sourcefile']})
        for doc in r
        ]

qaChain = load_qa_with_sources_chain(llm, chain_type=chainType)
#qaChain.run(input_documents=docs, question=query)
answer = qaChain({"input_documents": docs, "question": query}, return_only_outputs=True)
outputAnswer = answer['output_text']
print(outputAnswer)

c:\Users\astalati\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Microsoft Fabric is a unified platform that provides data and analytics solutions for organizations. It covers various aspects such as data movement, data science, real-time analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place. Fabric is built on a foundation of Software as a Service (SaaS) and aims to simplify analytics needs by providing an integrated and easy-to-use product. It brings together different experiences into a unified platform and offers a comprehensive big data analytics platform. Fabric includes components such as Data Engineering, Data Factory, and Data Science, each tailored to specific personas and tasks. It is built on top of ADLS (Azure Data Lake Storage) Gen2 and eliminates data silos. Fabric allows creators to focus on their work without the need to manage or understand the underlying infrastructure. 
SOURCES: Fabric Get Started.pdf


In [5]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import QAGenerationChain
from langchain.chains import LLMChain

# Let's generate followup questions
followupTemplate = """
    Generate three very brief questions that the user would likely ask next.
    Use double angle brackets to reference the questions, e.g. <What is Azure?>.
    Try not to repeat questions that have already been asked.  Don't include the context in the answer.

    Return the questions in the following format:
    <>
    <>
    <>
    
    ALWAYS return a "NEXT QUESTIONS" part in your answer.

    {context}
    """
followupPrompt = PromptTemplate(template=followupTemplate, input_variables=["context"])
# followupChain = load_qa_chain(llm, chain_type='stuff', prompt=followupPrompt)
# followupAnswer = followupChain({"input_documents": docs}, return_only_outputs=True)
# nextQuestions = followupAnswer['output_text'].replace("Answer: ", '').replace("Sources:", 'SOURCES:').replace("Next Questions:", 'NEXT QUESTIONS:').replace('NEXT QUESTIONS:', '').replace('NEXT QUESTIONS', '')
# print(nextQuestions)

rawDocs = ""
llm_chain = LLMChain(prompt=followupPrompt, llm=llm)
for doc in docs:
    rawDocs = rawDocs + "\\r\\n" + doc.page_content

llm_chain.predict(context=docs)

'<What is Microsoft Fabric?>\n<What are the components of Microsoft Fabric?>\n<What is the purpose of OneLake in Microsoft Fabric?>'

#### How about we ask a question for which the answer is not in the document we have indexed in Vector Store

In [7]:
chainType = "stuff"
topK = 3
query = "Tell me a Joke"
#query = "Who is the CEO of Microsoft"
# Since we already index our document, we can perform the search on the query to retrieve "TopK" documents
r = performCogSearch(OpenAiEndPoint, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey, embeddingModelType, OpenAiEmbedding, query, indexName, topK)

if r == None:
    docs = [Document(page_content="No results found")]
else :
    docs = [
        Document(page_content=doc['content'], metadata={"id": doc['id'], "source": doc['sourcefile']})
        for doc in r
        ]

qaChain = load_qa_with_sources_chain(llm, chain_type=chainType)
#qaChain.run(input_documents=docs, question=query)
answer = qaChain({"input_documents": docs, "question": query}, return_only_outputs=True)
outputAnswer = answer['output_text']
print(outputAnswer)


I'm sorry, I don't have the capability to tell jokes.


#### What if we don't want to have LLM answer the question outside of the document we have indexed in Vector Store. We can use the custom prompt to do that.

In [8]:
chainType = "stuff"
topK = 3
query = "Who is the CEO of Microsoft"

# Since we already index our document, we can perform the search on the query to retrieve "TopK" documents
r = performCogSearch(OpenAiEndPoint, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey, embeddingModelType, OpenAiEmbedding, query, indexName, topK)

if r == None:
    docs = [Document(page_content="No results found")]
else :
    docs = [
        Document(page_content=doc['content'], metadata={"id": doc['id'], "source": doc['sourcefile']})
        for doc in r
        ]

template = """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer. 
            If the answer is not contained within the text below, say \"I don't know\".

            QUESTION: {question}
            =========
            {summaries}
            =========
            """
#qaPrompt = load_prompt('lc://prompts/qa_with_sources/stuff/basic.json')
qaPrompt = PromptTemplate(template=template, input_variables=["summaries", "question"])
qaChain = load_qa_with_sources_chain(llm, chain_type=chainType, prompt=qaPrompt)
#qaChain.run(input_documents=docs, question=query)
answer = qaChain({"input_documents": docs, "question": query}, return_only_outputs=True)
outputAnswer = answer['output_text']
print(outputAnswer)

The answer to the question "Who is the CEO of Microsoft?" is not contained within the provided text.


#### Chain type
This category of chains are used for interacting with indexes. The purpose these chains is to combine your own data (stored in the indexes) with LLMs. The best example of this is question answering over your own documents.

A big part of this is understanding how to pass multiple documents to the language model. There are a few different methods, or chains, for doing so. LangChain supports four of the more common ones - and we are actively looking to include more, so if you have any ideas please reach out! Note that there is not one best method - the decision of which one to use is often very context specific. In order from simplest to most complex

##### Stuff
Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as the StuffDocumentsChain.

- Pros: Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.
- Cons: Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.

##### Map-Reduce
This method involves running an initial prompt on each chunk of data (for summarization tasks, this could be a summary of that chunk; for question-answering tasks, it could be an answer based solely on that chunk). Then a different prompt is run to combine all the initial outputs. This is implemented in the LangChain as the MapReduceDocumentsChain.

- Pros: Can scale to larger documents (and more documents) than StuffDocumentsChain. The calls to the LLM on individual documents are independent and can therefore be parallelized.
- Cons: Requires many more calls to the LLM than StuffDocumentsChain. Loses some information during the final combined call.

##### Refine
This method involves running an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document, asking the LLM to refine the output based on the new document.

- Pros: Can pull in more relevant context, and may be less lossy than MapReduceDocumentsChain.
- Cons: Requires many more calls to the LLM than StuffDocumentsChain. The calls are also NOT independent, meaning they cannot be paralleled like MapReduceDocumentsChain. There is also some potential dependencies on the ordering of the documents.

##### Map-Rerank
This method involves running an initial prompt on each chunk of data, that not only tries to complete a task but also gives a score for how certain it is in its answer. The responses are then ranked according to this score, and the highest score is returned.

- Pros: Similar pros as MapReduceDocumentsChain. Requires fewer calls, compared to MapReduceDocumentsChain.
- Cons: Cannot combine information between documents. This means it is most useful when you expect there to be a single simple answer in a single document.

##### Let's test the same question with Map Reduce Chaintype

In [9]:
topK = 3
query = "What is Microsoft Fabric"
chainType = "map_reduce"

# Since we already index our document, we can perform the search on the query to retrieve "TopK" documents
r = performCogSearch(OpenAiEndPoint, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey, embeddingModelType, OpenAiEmbedding, query, indexName, topK)

if r == None:
    docs = [Document(page_content="No results found")]
else :
    docs = [
        Document(page_content=doc['content'], metadata={"id": doc['id'], "source": doc['sourcefile']})
        for doc in r
        ]

qaTemplate = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
            Return any relevant text.
            {context}
            Question: {question}
            Relevant text, if any :"""

qaPrompt = PromptTemplate(
    template=qaTemplate, input_variables=["context", "question"]
)

combinePromptTemplate = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: {question}
=========
{summaries}
=========
"""
combinePrompt = PromptTemplate(
    template=combinePromptTemplate, input_variables=["summaries", "question"]
)

qaChain = load_qa_with_sources_chain(llm, chain_type=chainType, question_prompt=qaPrompt, 
                                     combine_prompt=combinePrompt, 
                                     return_intermediate_steps=True)
answer = qaChain({"input_documents": docs, "question": query})
outputAnswer = answer['output_text']
print(outputAnswer)

Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place. Fabric brings together various experiences and offers a comprehensive big data analytics platform, allowing organizations and individuals to turn large and complex data repositories into actionable workloads and analytics. It unifies the OneLake and lakehouse architecture across enterprises, providing a unified location to store all organizational data and simplifying the user experience. Fabric allows creators to concentrate on producing their best work without the need to integrate, manage, or understand the underlying infrastructure. It offers a comprehensive set of analytics experiences tailored to specific personas and tasks, including Data Engineering, Data Factory, and Data Science. D

In [10]:
# For the chaintype of MapReduce and Refine, we can also get insight into intermediate steps of the pipeline.
# This way you can inspect the results from map_reduce chain type, each top similar chunk summary
intermediateSteps = answer['intermediate_steps']
for step in intermediateSteps:
        display(HTML("<b>Chunk Summary:</b> " + step))

##### This time with Refine Chain Type

In [11]:
topK = 3
query = "What is Microsoft Fabric"
chainType = "refine"

# Since we already index our document, we can perform the search on the query to retrieve "TopK" documents
r = performCogSearch(OpenAiEndPoint, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey, embeddingModelType, OpenAiEmbedding, query, indexName, topK)

if r == None:
    docs = [Document(page_content="No results found")]
else :
    docs = [
        Document(page_content=doc['content'], metadata={"id": doc['id'], "source": doc['sourcefile']})
        for doc in r
        ]
    
refineTemplate = (
                    "The original question is as follows: {question}\n"
                    "We have provided an existing answer, including sources: {existing_answer}\n"
                    "We have the opportunity to refine the existing answer"
                    "(only if needed) with some more context below.\n"
                    "------------\n"
                    "{context_str}\n"
                    "------------\n"
                    "Given the new context, refine the original answer to better "
                    "If you do update it, please update the sources as well. "
                    "If the context isn't useful, return the original answer."
                )
refinePrompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refineTemplate,
)

qaTemplate = (
    "Answer the question as truthfully as possible using the provided text below, and if the answer is not contained within the text below, say \"I don't know\"\n"
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {question}\n"
    "\n---------------------\n"
)
qaPrompt = PromptTemplate(
    input_variables=["context_str", "question"], template=qaTemplate
)
qaChain = load_qa_with_sources_chain(llm, chain_type=chainType, question_prompt=qaPrompt, refine_prompt=refinePrompt,
                                     return_intermediate_steps=True)

answer = qaChain({"input_documents": docs, "question": query}, return_only_outputs=True)
modifiedAnswer = answer['output_text']
print(modifiedAnswer)

Microsoft Fabric is an all-in-one analytics solution for enterprises that brings together data movement, data science, real-time analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, data integration, and data science, all in one platform. Fabric allows creators to concentrate on producing their best work, freeing them from the need to integrate, manage, or understand the underlying infrastructure that supports the experience.

Fabric includes industry-leading experiences in the following categories for an end-to-end analytical need:

1. Data Engineering: The Data Engineering experience provides a world-class Spark platform with great authoring experiences, enabling data engineers to perform large-scale data transformation and democratize data through the lakehouse. Microsoft Fabric Spark's integration with Data Factory enables notebooks and Spark jobs to be scheduled and orchestrated.

2. Data Factory: Azure Data Fact

In [12]:
# For the chaintype of MapReduce and Refine, we can also get insight into intermediate steps of the pipeline.
# This way you can inspect the results from map_reduce chain type, each top similar chunk summary
intermediateSteps = answer['intermediate_steps']
for step in intermediateSteps:
        display(HTML("<b>Chunk Summary:</b> " + step))